# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

device_id = 1
bb.set_device(device_id)
print(f"BinaryBrain : {bb.get_version_string()}")
print(f"Device : {bb.get_device_name(device_id)}")

BinaryBrain : 4.3.1
Device : NVIDIA GeForce GT 1030


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*2], connection='random', N=4),
            bb.DifferentiableLut([64*4],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),
            bb.DifferentiableLut([10*4*4], connection='random', N=4),
            bb.DifferentiableLut([10*4],   connection='serial', N=4),
            bb.AverageLut       ([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam(learning_rate=0.001)

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [5]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

  0%|          | 0/938 [00:00<?, ?it/s]

epoch[0] : loss=1.670814 accuracy=0.771000


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[1] : loss=1.692337 accuracy=0.749700


  0%|          | 0/938 [00:00<?, ?it/s]

...

epoch[5] : loss=1.682730 accuracy=0.744000


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[6] : loss=1.684111 accuracy=0.765600


  0%|          | 0/938 [00:00<?, ?it/s]

epoch[7] : loss=1.696528 accuracy=0.750000


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="")

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut_simple/MnistLutSimple.v'

In [7]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [8]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、4つの入力と接続されており、layer0 の出力ノードは 256 個あるので、256x4 の行列が取得できます。

In [9]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(512, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [770, 570, 749, 161],
       [633, 361, 153, 380],
       [221, 664, 145, 316]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [10]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(512, 16)


array([[False, False,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [False,  True,  True, ...,  True, False, False],
       ...,
       [ True,  True, False, ..., False,  True,  True],
       [ True,  True, False, ..., False, False, False],
       [ True, False,  True, ..., False, False, False]])

### 重み行列を覗いてみる

4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [11]:
W = layer0.W().numpy()
print(W.shape)
W

(512, 16)


array([[0.42467824, 0.38932997, 0.4948863 , ..., 0.53853285, 0.5638993 ,
        0.54345065],
       [0.49388063, 0.5786629 , 0.49178162, ..., 0.49223968, 0.48021594,
        0.5018154 ],
       [0.47597328, 0.5563937 , 0.63337225, ..., 0.62447244, 0.48365352,
        0.44047454],
       ...,
       [0.56074023, 0.62791616, 0.43050757, ..., 0.4585622 , 0.5561211 ,
        0.5656052 ],
       [0.51154643, 0.53712136, 0.47106028, ..., 0.45823413, 0.4095134 ,
        0.47624946],
       [0.5533829 , 0.44380018, 0.54016584, ..., 0.5313832 , 0.49502274,
        0.4817487 ]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [12]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(512, 16)


array([[ 3.2111042e-04,  1.0703686e-04, -5.0341024e-04, ...,
         5.4637734e-05, -1.0314485e-04, -3.4381606e-05],
       [ 1.0244548e-08,  4.6566129e-09,  4.6566129e-09, ...,
        -1.9208528e-09, -1.9208528e-09, -6.0390448e-10],
       [ 7.3739910e-04,  2.4197978e-04, -9.6131174e-05, ...,
        -1.1196680e-05, -1.7634559e-04, -7.0240872e-05],
       ...,
       [-4.2470219e-04, -1.4173199e-04,  6.5805833e-04, ...,
        -6.7224668e-05,  1.2389856e-04,  4.1293442e-05],
       [-1.3566634e-05,  7.3785317e-04, -6.3379412e-06, ...,
        -2.9338646e-04,  1.5495019e-04, -1.0324221e-04],
       [ 4.8534479e-05,  1.6178237e-05, -6.1726314e-06, ...,
        -1.9620493e-06, -8.2329789e-06, -2.7442838e-06]], dtype=float32)